In [3]:
import pandas as pd
import numpy as np
import math
import pickle
from datetime import datetime

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

# import matlab.engine as engi
# import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

In [16]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    
    release_df = pd.read_pickle('data/release/' + project + '_release.pkl')
    release_df = release_df.sort_values('created_at',ascending=False)
    release_df = release_df.reset_index(drop=True)
#     df_test_releases = release_df[0:5]
#     df_test_releases.reset_index(inplace = True, drop=True)
#     df_train_releases = release_df[5:]
#     df_train_releases.reset_index(inplace = True, drop=True)
#     last_train_release = datetime.strptime(df_train_releases.loc[0,'created_at'], '%Y-%m-%d').date()
    
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
            
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    commit_guru_df['created_at'] = pd.to_datetime(commit_guru_df.author_date_unix_timestamp,unit='s')
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes',],axis=1)

    
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df.merge(commit_guru_df,on='commit_hash')
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
#     df.to_csv('data/converted/'+ project + '_understand.csv',index=False)
    y = df.Bugs
    df = df.drop('Bugs',axis = 1)
    cols = df.columns
    created_at = df.created_at
    df = df.drop('created_at',axis = 1)
    scaler = MinMaxScaler()
    df = scaler.fit_transform(df)
    df = pd.DataFrame(X,columns = cols[:-1])
    df['created_at'] = created_at
    df['Bugs'] = y
    
#     df_last_commit_date = df.loc[df.shape[0],'created_at']

    df_last_commit_date = df.loc[df.shape[0]-1,'created_at']
    last_train_date = None
    count = 0
    for i in range(release_df.shape[0]):
        release_date = datetime.strptime(release_df.loc[i,'created_at'], '%Y-%m-%d')
        if release_date <= df_last_commit_date:
            count += 1
        if count == 5:
            last_train_date = release_date
            break
    
#     print(last_train_date)
#     print(df)
    
    if last_train_date == None:
        return df,df,0
    
    
    
    df['created_at'] = pd.to_datetime(df.created_at,unit='s')
    train_df =  df[df.created_at < last_train_date]
    test_df =  df[df.created_at >= last_train_date]
    
    train_df = train_df.drop('created_at',axis = 1)
    test_df = test_df.drop('created_at',axis = 1)
    
    if train_df.shape[0] == 0:
        return df,df,0
    
    if test_df.shape[0] == 0:
        return df,df,0
    
    return train_df,test_df,1


def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [17]:
def run_self(project):
    train_df, test_df,complete = load_data(project)
    if complete == 0:
        return None,None,None,None,None,None,None,None,None
    loc = test_df.CountLineCode
    df_smote = train_df
    df_smote = apply_smote(df_smote)
#     df_smote,cols = apply_cfs(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf =  RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = 0
    
#     test_df = test_df[cols]
    y_test = test_df.Bugs
    X_test = test_df.drop('Bugs',axis = 1)
    
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [18]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()[150:]
print(len(projects))

885


In [20]:
count = 0
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
featue_importance = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
#         if project != 'redis-manager':
#             continue
#         if df.shape[0] > 10:
        recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance = run_self(project)
        print('here')
        if recall == None:
            continue
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
        featue_importance[project] = importance
    except ValueError as e:
        print(project,e)
        continue

NameError: name 'X' is not defined

In [155]:
for project in projects:
    if project == '.DS_Store':
        continue
    if project != 'maven-plugins':
        continue
    run_self(project)


In [164]:
np.nanmedian(list(precision_list.values())),np.nanmedian(list(recall_list.values())),np.nanmedian(list(pf_list.values()))

(0.42, 0.36, 0.37)